# Download historical weather for points contained inside a GPX file

## Steps

- Parse a GPX file into a Pandas DataFrame
- Calculate the distance between points
- Interpolate if needed
- Generate download JSON

## Parameters & dependences

In [1]:
# Dependences
import requests as req
import gpxpy
from geopy.distance import geodesic
import pandas as pd
import numpy as np
from scipy.optimize import brentq
import plotly.express as px
from tqdm.auto import tqdm
from typing import List, Tuple, Dict

In [2]:
# Parameters
route_path = '../data/uiramata_chui.gpx'

## Load GPX

In [3]:
def calculate_deltas(df: pd.DataFrame) -> Dict[int, Tuple[float, float, float]]:
    """
    
    """
    deltas = {}
    for i, (ind, row) in tqdm(enumerate(df.iterrows()),
                              total=len(df),
                              desc='Calculating deltas'):
        (lat2, lon2) = (row.lat, row.lon)        
        if i > 0:
            # Calculate horizontal, vertical and actual deltas
            delta_x = geodesic((lat1, lon1), (lat1, lon2)).meters
            delta_y = geodesic((lat1, lon1), (lat2, lon1)).meters
            delta = geodesic((lat1, lon1), (lat2, lon2)).meters
            
            # Put a negative signal if it is west / south directed
            if lon2 < lon1:
                delta_x *= -1
            if lat2 < lat1:
                delta_y *= -1
            
            # Attribute tuple of deltas 
            deltas[ind] = (delta, delta_x, delta_y)
        (lat1, lon1) = (lat2, lon2)
    return deltas


def load_route(path: str) -> pd.DataFrame:
    """
    
    """
    # Load and parse GPX
    with open(path, 'r') as fid:
        content: str = fid.read()
    gpx: dict = gpxpy.parse(content)
    
    # Get points associated with the first track and segment
    route_points: List[Tuple[float, float, float]] = gpx.tracks[0].segments[0].points
    
    # Generate dataframe from points
    cols = ['lon', 'lat', 'ele']
    df = (pd.DataFrame([(p.longitude, p.latitude, p.elevation) 
                         for p 
                         in route_points],
                       columns=cols)
          .drop_duplicates(subset=['lat', 'lon'])
          )
    return df


def append_deltas(df: pd.DataFrame) -> pd.DataFrame:
    """
    
    """
    # Get the distance difference between each sucessive point
    deltas = calculate_deltas(df)
    
    # Put deltas into variables
    delta = {k: v[0] for k, v in deltas.items()}
    delta_x = {k: v[1] for k, v in deltas.items()}
    delta_y = {k: v[2] for k, v in deltas.items()}
    
    # Append deltas        lon2 = row.lon

    df = df.join(pd.Series(delta, name='delta'))
    df = df.join(pd.Series(delta_x, name='delta_x'))
    df = df.join(pd.Series(delta_y, name='delta_y'))
    
    # Calculate velocities on the latitude and longitude axes
    df = df.assign(u_x=lambda df: df.delta_x / df.delta)
    df = df.assign(u_y=lambda df: df.delta_y / df.delta)
    
    return df

In [6]:
df = (load_route(route_path).pipe(append_deltas))

In [10]:
df = df.assign(cumm_km=lambda df: df.delta.cumsum() / 1000)

In [11]:
intervals = df.cumm_km // 1
inds = intervals.diff() != 0.0
interval_points = intervals[inds].index
points = df.loc[interval_points]

In [13]:
df.lon.max()

-52.31065

In [14]:
df.lon.min()

-63.99714

In [15]:
df.lat.max()

4.59536

In [17]:
df.lat.min()

-33.74696